In [ ]:
# TODO: Replace the string "Your Name" and "Date of measurement" with the appropriate values.

from header import header
_ = header(student="Your Name", date="Date of measurement", task_no=4)

$\color{red}{\textbf{DŮLEŽITÉ}}$ Pokud nemáte nainstalované všechny potřebné knihovny, spusťte následující buňku:

In [ ]:
!pip install numpy matplotlib scipy pandas uncertainties

$\color{red}{\textbf{DŮLEŽITÉ}}$ Pokud nemáte všechny potřebné vstupní soubory lokálně, můžete si je stáhnout z GitHubu: https://github.com/vojtechpleskot/physicslab4/tree/main/A4 

Pokud spouštíte tento notebook v Google Colab nebo podobném prostředí, můžete soubory stáhnout spuštěním následující buňky:

In [ ]:
# Files with theory predictions.
!wget https://raw.githubusercontent.com/vojtechpleskot/physicslab4/main/A4/Pb_theory_all.txt
!wget https://raw.githubusercontent.com/vojtechpleskot/physicslab4/main/A4/Pb_attenuation_length.txt
!wget https://raw.githubusercontent.com/vojtechpleskot/physicslab4/main/A4/Cd_attenuation_length.txt
!wget https://raw.githubusercontent.com/vojtechpleskot/physicslab4/main/A4/Fe_attenuation_length.txt

# To store files containing data, create a directory.
!mkdir P-IV_A4_001

!wget -P P-IV_A4_001 https://raw.githubusercontent.com/vojtechpleskot/physicslab4/main/A4/P-IV_A4_001/0.dat
!wget -P P-IV_A4_001 https://raw.githubusercontent.com/vojtechpleskot/physicslab4/main/A4/P-IV_A4_001/Cd_0106.dat
!wget -P P-IV_A4_001 https://raw.githubusercontent.com/vojtechpleskot/physicslab4/main/A4/P-IV_A4_001/Cd_0214.dat
!wget -P P-IV_A4_001 https://raw.githubusercontent.com/vojtechpleskot/physicslab4/main/A4/P-IV_A4_001/Cd_0322.dat
!wget -P P-IV_A4_001 https://raw.githubusercontent.com/vojtechpleskot/physicslab4/main/A4/P-IV_A4_001/Cd_0428.dat
!wget -P P-IV_A4_001 https://raw.githubusercontent.com/vojtechpleskot/physicslab4/main/A4/P-IV_A4_001/Fe_0500.dat
!wget -P P-IV_A4_001 https://raw.githubusercontent.com/vojtechpleskot/physicslab4/main/A4/P-IV_A4_001/Fe_1092.dat
!wget -P P-IV_A4_001 https://raw.githubusercontent.com/vojtechpleskot/physicslab4/main/A4/P-IV_A4_001/Fe_1578.dat
!wget -P P-IV_A4_001 https://raw.githubusercontent.com/vojtechpleskot/physicslab4/main/A4/P-IV_A4_001/Fe_2074.dat
!wget -P P-IV_A4_001 https://raw.githubusercontent.com/vojtechpleskot/physicslab4/main/A4/P-IV_A4_001/Pb_0530.dat
!wget -P P-IV_A4_001 https://raw.githubusercontent.com/vojtechpleskot/physicslab4/main/A4/P-IV_A4_001/Pb_0650.dat
!wget -P P-IV_A4_001 https://raw.githubusercontent.com/vojtechpleskot/physicslab4/main/A4/P-IV_A4_001/Pb_0762.dat
!wget -P P-IV_A4_001 https://raw.githubusercontent.com/vojtechpleskot/physicslab4/main/A4/P-IV_A4_001/Pb_0872.dat

# Celkový absorpční koeficient gama záření

# Úvod

Proces zeslabení svazku gama záření v prostředí nelze považovat za spojitý, jak je tomu u těžkých nabitých částic. Při interakci fotonu s elektrony atomového obalu prostředí buď foton ztratí veškerou svou energii fotoelektrickým jevem nebo tvorbou elektron-pozitronových párů, nebo její část Comptonovým (nekoherentním) rozptylem. Tvorba elektron-pozitronových párů může přirozeně probíhat také v poli jádra. Při koherentním rozptylu foton nedeponuje energii, ale mění směr. Při nekoherentním rozptylu kromě ztráty energie také mění směr. Důsledkem těchto interakcí pro experiment je zeslabení svazku gama záření při průchodu vrstvou materiálu tloušťky $t$ podle vztahu:
$$
N = N_0 e^{-\mu t},
$$
kde $N_0$ je počet gama kvant dopadajících na vrstvu materiálu tloušťky $t$, $N$ je počet fotonů s původní energií za vrstvou materiálu, $\mu$ je absorpční koeficient nebo lineární zeslabovací koeficient s rozměrem $\mathrm{mm}^{-1}$.
Protože závislost zeslabení svazku gama záření na tloušťce materiálu má exponenciální charakter, vždy existuje nenulová pravděpodobnost, že gama kvantum projde jakoukoli tloušťkou jakéhokoli materiálu bez interakce.
Pro každou danou energii dopadajících fotonů $E_\gamma$ se tak může volná dráha jednotlivých fotonů významně lišit od střední volné dráhy $1/\mu(E_\gamma)$.

Interakce elektromagnetického záření s hmotou je velmi dobře známa. Jednotlivé účinné průřezy jako funkce energie fotonu jsou mimo jiné tabelovány. Závislost absorpčního koeficientu $\mu$ na energii dopadajících fotonů $E_\gamma$ je ukázána pro olovo na níže uvedeném obrázku (stačí spustit python kód). Pro konkrétní experimenty je často praktičtější určit zeslabení svazku experimentálně (pro konkrétní detektor, geometrii, ...) a případně jej ověřit pomocí počítačové simulace metodou Monte Carlo.
Pro interakci fotonů s materiálem má také nenulový účinný průřez "jaderný fotoelektrický jev" (tj. foto-jaderné reakce - např. $(\gamma, n)$). Účinný průřez těchto reakcí je však zanedbatelný až do počáteční energie fotonu řádově desítky MeV ve srovnání s účinnými průřezy výše zmíněných interakcí.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read in the theory values, and multiply them by the density of the material.
# Convert units of E to keV and attenuation length to mm^-1.

# Read.
df = pd.read_csv('Pb_theory_all.txt', skiprows=3, sep='|', names=['E', 'sigma_R', 'sigma', 'tau', 'kappa_n', 'kappa_e', 'w', 'wo', ''])

# Parameter scaling by density and conversion to mm^-1
scale = 11.34 / 10

# Convert energy to keV
df['E']  *= 1000

# Calculate the total electron-positron pair production cross section.
df['kappa_tot'] = df['kappa_n'] + df['kappa_e']

# Plot the curves.
plt.plot(df['E'], df['w'] * scale, label=r'$\mu_\mathrm{with}$'   , color='black', linestyle='-')
plt.plot(df['E'], df['wo'] * scale, label=r'$\mu_\mathrm{without}$', color='red', linestyle='-')
plt.plot(df['E'], df['tau'] * scale, label=r'$\tau$', color='gray', linestyle='-.')
plt.plot(df['E'], df['sigma_R'] * scale, label=r'$\sigma_\mathrm{R}$', color='blue', linestyle=':')
plt.plot(df['E'], df['sigma'] * scale, label=r'$\sigma$', color='purple', linestyle='--')
plt.plot(df['E'], df['kappa_tot'] * scale, label=r'$\kappa$', color='green', linestyle='--')
plt.xlabel(r'$E_\gamma\ [\mathrm{keV}]$')
plt.ylabel(r'$\mu\ [\mathrm{mm}^{-1}]$')
plt.title(f'Pb, Attenuation coefficient')
plt.legend(ncol=2)
plt.yscale('log')
plt.ylim(1e-4, 1e2)
plt.show()

Výše uvedený obrázek ukazuje závislost absorpčního koeficientu $\mu$ na energii fotonu $E_\gamma$ pro olovo. Graf rozlišuje jednotlivé procesy přispívající k zeslabení fotonového svazku v měřeném energetickém rozsahu, konkrétně fotoelektrický jev $\tau$, koherentní nebo Rayleighův rozptyl $\sigma_\mathrm{R}$, Comptonův rozptyl $\sigma$ a tvorbu elektron-pozitronových párů $\kappa$. Celkové zeslabení svazku je pak popsáno dvěma křivkami - se zahrnutím koherentního rozptylu jako zeslabujícího procesu $\mu_\mathrm{with}$ a bez něj, $\mu_\mathrm{without}$. Křivky jsou interpolací hodnot z https://www.nist.gov/pml/xcom-photon-cross-sections-database

# Experimentální sestava

Jako zdroj fotonů se používá emitor $^{226}\mathrm{Ra}$. Počet přenesených fotonů jako funkce energie se měří pomocí polovodičového germaniového detektoru připojeného k počítači s operačním softwarem. Štěrbina ve stojanu mezi emitorem a detektorem slouží k vkládání vrstev jednotlivých materiálů.


# Měření

Doba měření pro každý materiál je typicky nastavena na 250 nebo 300 sekund. Měření probíhá v následujících krocích:
- Spektrum $^{226}\mathrm{Ra}$ se měří bez jakéhokoli stínění.
  - Výsledné spektrum se používá pro kalibraci spektrometru. Energie intenzivních linií $^{226}\mathrm{Ra}$ jsou dobře známy a jsou přiřazeny odpovídajícím pozicím píků ve spektru.
- Provede se dvanáct dalších měření spektra:
  - Pro čtyři různé tloušťky tří různých materiálů - Cd, Pb, Fe.
    - U každého materiálu je k dispozici několik destiček, které lze postupně navršit do štěrbiny mezi radioaktivní zdroj a detektor.
  - Díky tomu získáme měření spektra pro pět různých tlouštěk stínění pro každý materiál, protože je zahrnuto také měření bez stínění (tj. tloušťka 0 mm), které je společné pro všechny materiály.
- V každém spektru je identifikováno 15 píků a je určen počet událostí v každém píku.
  - Počet událostí v každém píku je určen softwarem používaným k ovládání spektrometru.
    - Počet událostí v každém píku je korigován na pozadí.
  - Software také poskytuje energii každého píku.
  - Počet událostí v každém píku klesá exponenciálně s tloušťkou stínění.
    - Připomeňme si vzorec $N = N_0 e^{-\mu t}$.
    - Připomeňme si, že zeslabovací koeficient je funkcí energie fotonů: $\mu = \mu(E_\gamma)$.
      - Předem definovaných 15 píků odpovídá 15 různým energiím fotonů. Můžeme tak současně měřit zeslabovací koeficient pro 15 různých energií.
  - 15 píků je předem definováno na základě jejich intenzity. Pro využití více píků ve spektru $^{226}\mathrm{Ra}$ bychom potřebovali delší než 300sekundová měření.
- V každém spektru je také všudypřítomný pík 1460 keV z rozpadu $^{40}\mathrm{K}$.
  - $^{40}\mathrm{K}$ je běžný zdroj pozadí.
  - Jako pozadí přicházejí fotony z rozpadů $^{40}\mathrm{K}$ k detektoru ze všech směrů. Proto počet událostí v píku 1460 keV není ovlivněn tloušťkou stínění.

$\color{red}{\textbf{Úkol:}}$ Načtěte naměřená data z poskytnutých textových souborů. Vytiskněte data odpovídající stavu bez stínění jako tabulku. Data jsou souhrnné informace o 15 předem definovaných pících. Pro každý pík je uvedena:
- energie píku,
- počet událostí v píku.

In [ ]:
# Insert your code here

import pandas as pd
import numpy as np

# Read in the data from the .dat file and store it in a pandas dataframe.
# The data is stored in format separated by blank spaces.
# Preprocess the file so that each row with less than 7 columns is filled with NaN values.
# For preprocessing, read the file line by line and split the line by the separator.
def read_data(file):
    with open(file, 'r') as f:
        lines = f.readlines()

    # Write the updated lines to a new file
    with open(file + '.2', 'w') as f:
        for line in lines:
            row = line.split()
            if len(row) < 7:
                row += ["NaN"] * (7 - len(row))
            f.write(','.join(row) + '\n')
    
    # Store the data in a pandas dataframe
    data = pd.read_csv(file + '.2', sep=',', header=None, skiprows=1)

    # Keep just the first 7 columns of the dataframe
    data = data.iloc[:, 0:7]

    # Rename the columns of the dataframe to the following:
    data.columns = ['PN', 'E', 'FWHM', 'N', 'B', 'Net C/S', '%Err']

    return data

    
# Read the data corresponding to no shielding.
data0 = read_data('P-IV_A4_001/0.dat')

# Print data0 to see whether the format is correct.
print(data0)

# Read in data from the following files in exactly the same way:
#              Cd_0214.dat  Cd_0428.dat  Fe_0500.dat  Fe_1578.dat  Pb_0530.dat  Pb_0762.dat  
# Cd_0106.dat  Cd_0322.dat  Fe_1092.dat  Fe_2074.dat  Pb_0650.dat  Pb_0872.dat  
# All dataframes are stored in a dictionary with the keys being Cd, Fe, Pb.
# The values are dictionaries: {thickness: dataframe, thickness: dataframe, ...}
# where thickness is the thickness of the shielding in 10e-5 m and dataframe is the dataframe.
# The thickness is the number in the file name before the .dat extension.

files = ['P-IV_A4_001/Cd_0106.dat', 'P-IV_A4_001/Cd_0214.dat', 'P-IV_A4_001/Cd_0322.dat', 'P-IV_A4_001/Cd_0428.dat', 'P-IV_A4_001/Fe_0500.dat', 'P-IV_A4_001/Fe_1092.dat', 'P-IV_A4_001/Fe_1578.dat', 'P-IV_A4_001/Fe_2074.dat', 'P-IV_A4_001/Pb_0530.dat', 'P-IV_A4_001/Pb_0650.dat', 'P-IV_A4_001/Pb_0762.dat', 'P-IV_A4_001/Pb_0872.dat']
data_dict = {}
for file in files:
    element = file.split('/')[-1].split('_')[0]
    thickness = int(file.split('_')[-1].split('.')[0]) / 100.
    data = read_data(file)
    if element not in data_dict:
        data_dict[element] = {0. : data0} # Store the data with no shielding to the first element of the list
    data_dict[element][thickness] = data


$\color{red}{\textbf{Úkol:}}$ Načtěte teoretické hodnoty zeslabovacích koeficientů gama záření ve Fe, Cd a Pb z textových souborů, které byly poskytnuty online databází https://www.nist.gov/pml/xcom-photon-cross-sections-database Všimněte si, že zeslabovací koeficienty jsou uvedeny v $\mathrm{cm}^2/\mathrm{g}$. Vynásobte je hustotou materiálu, abyste získali zeslabovací koeficient v $\mathrm{cm}^{-1}$. Převeďte jednotky na $\mathrm{mm}^{-1}$. Vykreslte teoretickou závislost zeslabovacího koeficientu na energii gama záření pro každý materiál. Graf by měl obsahovat dvě křivky pro každý materiál - se zahrnutím koherentního rozptylu jako zeslabujícího procesu a bez něj.

In [ ]:
# Insert your code here

import pandas as pd
import matplotlib.pyplot as plt

def read_attenuation_length(file):
    return pd.read_csv(file, skiprows=3, sep='|', names=['E', 'w', 'wo', ''])

# Read in the data to the theory dictionary, and multiply by the density of the material.
# Convert units of E to keV and attenuation length to mm^-1.
density = {'Cd': 8.65, 'Fe': 7.86, 'Pb': 11.34}
theory = {}
for element in data_dict:
    df = read_attenuation_length(f'{element}_attenuation_length.txt')
    df['E']  *= 1000 # Convert to keV
    df['w']  *= density[element] / 10 # Scale by density and convert to mm^-1
    df['wo'] *= density[element] / 10 # Scale by density and convert to mm^-1
    theory[element] = df

# Plot the curves.
for element in theory:
    df = theory[element]
    plt.plot(df['E'], df['w'] , label='With c. s.'   , color='black')
    plt.plot(df['E'], df['wo'], label='Without c. s.', color='red')
    plt.xlabel('Energy [keV]')
    plt.ylabel(r'$\mu\ [\mathrm{mm}^{-1}]$')
    plt.title(f'{element}, Attenuation coefficient')
    plt.legend()
    plt.show()

Pro vykreslování dat spolu s fitovanou křivkou může být užitečná následující python funkce:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

def plot(x, y, y_err, x_fit, y_fit, y_fit_lower, y_fit_upper, xlabel, ylabel, title = ''):

    # draw x, y with error bars
    plt.errorbar(x, y, y_err, fmt='o', label='Data', color='black')

    # draw the fit function and its uncertainty band
    plt.fill_between(x_fit, y_fit_lower, y_fit_upper, color='red', alpha=0.3)

    # create a legend entry for the fit function and its uncertainty band
    line_with_band = mpl.lines.Line2D([], [], color='red', label='Fit', linestyle='-', linewidth=2)
    band = mpl.patches.Patch(color='red', alpha=0.3, label='Fit uncertainty')

    # get the current legend handles and labels
    handles, labels = plt.gca().get_legend_handles_labels()
    plt.legend(handles=handles + [(line_with_band, band)], labels=labels + ['Fit'])

    # finally, plot
    plt.plot(x_fit, y_fit, 'r-')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    if title != '':
        plt.title(title)
    plt.show()

    return

$\color{red}{\textbf{Úkol:}}$ Pro každý stínicí materiál (Fe, Cd, Pb) a pro každý pík určete zeslabovací koeficient $\mu$:
- Provedněte fit grafu počtu událostí v píku, $N$, proti tloušťce stínění.
  - V grafu je pět bodů: bez stínění a čtyři různé tloušťky stínění.
  - Fitovací funkce je $N = N_0 e^{-\mu t}$.
  - Vykreslete data a fitovanou křivku pro dva vámi vybrané píky pro každý materiál.
- Uložte zeslabovací koeficient $\mu$ a odpovídající energii pro každý pík.
  - Např. do slovníku s energií jako klíčem a zeslabovacím koeficientem jako hodnotou.

In [ ]:

# Do the following for each element in the dictionary:
# - Create a new dataframe with the following columns:
#   - PN
#   - E
#   - mu
#   - mu_unc
# mu is the linear attenuation coefficient in mm^-1 and mu_unc is the uncertainty on mu.
# mu is evaluated in a fit of the form N = N0 * exp(-mu * thickness)
# where N is the net counts in a peak, N0 is the net counts with no shielding, and thickness is the thickness of the shielding.
# The uncertainty in N, estimated as sqrt(N), is taken into account in the fit.
# The uncertainty in mu is calculated in the fitting algorithm.

import numpy as np
from scipy.optimize import curve_fit
import uncertainties

def fit_func(x, mu, N0):
    return N0 * np.exp(-mu * x)

# Store the fit results - the mu values and their uncertainties - in a dictionary together with the corresponding energies.
fit_results = {element : {} for element in data_dict}

for element in data_dict:
    for irow in range(16):
        if irow == 10: # Skip the row with the potassium 40 background peak
            continue
        thickness = np.array(sorted(data_dict[element].keys()))
        N         = np.array([data_dict[element][t].loc[irow, 'N'] for t in thickness])

        try:
            nom, cov = curve_fit(fit_func, thickness, N, sigma=np.sqrt(N), absolute_sigma=True)
            mu = nom[0]
            mu_unc = np.sqrt(cov[0, 0])
        except:
            mu = np.nan
            mu_unc = np.nan
    
        # Store the fit result
        energy = data_dict[element][0.].loc[irow, 'E']
        fit_results[element][energy] = uncertainties.ufloat(mu, mu_unc)

        # Prepare the fit result plotting
        x = np.linspace(thickness.min(), thickness.max(), 100)
        mu, N0 = uncertainties.correlated_values(nom, cov)
        wrapped_exp = uncertainties.wrap(fit_func)
        y_fit_list = [wrapped_exp(x, mu, N0) for x in x]
        y_fit      = np.array([y.nominal_value for y in y_fit_list])
        sigma_fit  = np.array([y.std_dev       for y in y_fit_list])

        # Plot the data and the fit function with its uncertainty band
        title = f'Element: {element}, Energy: {data_dict[element][0.].loc[irow, "E"]} keV'
        if irow in [8, 15]:
            plot(thickness, N, np.sqrt(N), x, y_fit, y_fit - sigma_fit, y_fit + sigma_fit, 'Thickness [mm]', 'Net Counts', title)
            print(f'{title}, Mu: {mu} mm^-1')




$\color{red}{\textbf{Úkol:}}$ Vytiskněte tabulku se zeslabovacími koeficienty pro každou energii a každý materiál. Zahrňte také poloviční vrstvu (HVL), $t_{1/2}$, pro každou energii a každý materiál. HVL je definována jako tloušťka materiálu, která sníží intenzitu svazku gama záření na polovinu jeho počáteční hodnoty. HVL lze vypočítat jako:
$$
t_{1/2} = \frac{\ln(2)}{\mu}.
$$

In [ ]:
# Python DataFrame is suitable for presenting the results.

df = pd.DataFrame(fit_results)
df = df.rename_axis('Energy [keV]').reset_index()

for element in data_dict:
    df[f't_{element} [mm]'] = np.log(2) / df[element]

df = df.rename(columns={'Cd': 'mu_Cd [mm^-1]', 'Fe': 'mu_Fe [mm^-1]', 'Pb': 'mu_Pb [mm^-1]'})
print(df)


$\color{red}{\textbf{Úkol:}}$ Pro každý materiál vykreslete zeslabovací koeficient jako funkci energie gama záření.
- Na stejný graf vykreslete teoretické hodnoty zeslabovacích koeficientů.
  - Dvě křivky: se zahrnutím koherentního rozptylu jako zeslabujícího procesu a bez něj.
- Naměřené hodnoty zobrazte jako body a jejich nejistoty jako svislé chybové úsečky.

In [ ]:
# Insert your code here

for element in fit_results:

    energies = np.array(sorted(fit_results[element].keys()))
    # Remove elements around 1460 from energies (the potassium 40 peak)
    energies = np.delete(energies, np.where((energies > 1450) & (energies < 1470)))

    # Plot the data.
    mus = np.array([fit_results[element][e].nominal_value for e in energies])
    mus_unc = np.array([fit_results[element][e].std_dev for e in energies])
    plt.errorbar(energies, mus, mus_unc, fmt='o', label='Data', color='black')

    # Plot the theory curves.
    df = theory[element]
    plt.plot(df['E'], df['w'] , label='With c. s.'   , color='black')
    plt.plot(df['E'], df['wo'], label='Without c. s.', color='red')

    # Labels etc.
    plt.xlabel('Energy [keV]')
    plt.ylabel(r'$\mu\ [\mathrm{mm}^{-1}]$')
    plt.title(f'{element}')
    plt.legend()
    x = np.linspace(energies.min(), energies.max(), 100)
    plt.show()


$\color{red}{\textbf{Úkol:}}$ Diskutujte výsledky:
- Porovnejte naměřené a teoretické hodnoty zeslabovacích koeficientů.
- Diskutujte určení nejistoty naměřených hodnot.
  - Jsou některé efekty při určování nejistoty zanedbány?
    - Diskutujte vliv geometrie aparatury na výsledky měření (ve vztahu k možným systematickým trendům v naměřených hodnotách).

$\color{red}{\textbf{Řešení:}}$ 

$\color{red}{\textbf{Úkol:}}$ Pro pozaďový pík $^{40}\mathrm{K}$ (1460 keV):
- Vytvořte graf počtu událostí v píku jako funkci tloušťky stínění.
  - Zobrazte data ze všech měření (Fe, Cd, Pb) ve stejném grafu.
- Fitujte graf konstantní funkcí.

In [ ]:
# Insert your code here

# Check the index of the row with the potassium 40 peak, based on the energy.
idx = data_dict['Cd'][0.][data_dict['Cd'][0.]['E'].between(1450, 1470)].index[0]

# Check if we found the correct peak by printing its energy
print(f"Found peak at energy {data_dict['Cd'][0.].loc[idx, 'E']} keV")

# Plot N vs thickness for the potassium 40 peak for each material
for element in data_dict:
    thickness = np.array(sorted(data_dict[element].keys()))
    N = np.array([data_dict[element][t].loc[idx, 'N'] for t in thickness])
    plt.errorbar(thickness, N, yerr=np.sqrt(N), fmt='o', label=element)

# Fit with a constant function
def fit_func(x, c):
    return c * np.ones_like(x)

# Combine all data points for the fit
all_N = []
all_thickness = []
all_N_err = []
for element in data_dict:
    thickness = np.array(sorted(data_dict[element].keys()))
    N = np.array([data_dict[element][t].loc[idx, 'N'] for t in thickness])
    all_N.extend(N)
    all_thickness.extend(thickness)
    all_N_err.extend(np.sqrt(N))

all_N = np.array(all_N)
all_thickness = np.array(all_thickness)
all_N_err = np.array(all_N_err)

# Do the fit
nom, cov = curve_fit(fit_func, all_thickness, all_N, sigma=all_N_err, absolute_sigma=True)
c = uncertainties.ufloat(nom[0], np.sqrt(cov[0, 0]))

# Plot the fit result
x = np.linspace(0, max(all_thickness), 100)
plt.plot(x, fit_func(x, c.nominal_value), 'r-')

# draw the fit function uncertainty band
plt.fill_between(x, fit_func(x, c.nominal_value - c.std_dev), fit_func(x, c.nominal_value + c.std_dev), color='red', alpha=0.3)

plt.xlabel('Thickness [mm]')
plt.ylabel('Net Counts')
plt.title('K-40 peak (1461 keV)')


# create a legend entry for the fit function and its uncertainty band
line_with_band = mpl.lines.Line2D([], [], color='red', label='Fit', linestyle='-', linewidth=2)
band = mpl.patches.Patch(color='red', alpha=0.3, label='Fit uncertainty')

# get the current legend handles and labels
handles, labels = plt.gca().get_legend_handles_labels()
plt.legend(handles=handles + [(line_with_band, band)], labels=labels + ['Fit'])

plt.show()

# Print the fit result
print(f'K-40 peak (1461 keV) fit result: c = {c}')


$\color{red}{\textbf{Úkol:}}$ Diskutujte výsledky získané pro pozaďový pík $^{40}\mathrm{K}$ (1461 keV).

$\color{red}{\textbf{Řešení:}}$